Colab para generar clase_ternaria y obtener el archivo competencia_02.csv.gz para subir al bucket de google cloud (extraido de la ayudita que dio el profe Alejandro en Zullip)

## IMPORTANTE: SUBIR EL ARCHIVO competencia_02_crudo.csv.gz al entorno del Google Colab

In [2]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

"A diferencia de la competencia 1, ahora vamos a usar una base de datos en disco. Todo va a ser más lento, pero se va a poder trabajar sin problemas en máquinas de menor capacidad. La DB va a queda en el archivo dmeyf.db, lo probé en un colab básico y funcionó sin problemas."


In [3]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:////content/dmeyf.db

"Cargamos el dataset, y como tenemos la base en el disco, no debemos cargarlo más, la próxima se puede usar directamente"


In [8]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('/content/competencia_02_crudo.csv.gz')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


"El ayudin. Que hace el siguiente código:

Toma todos los valores de foto_mes y numero_de_cliente
Cruza todos con todos. De esta manera nos aseguramos que va haber un registro para todos los clientes en todos los periodos.
Luego se crean las variables soporte mes_0, que nos dice si el cliente esta en esa foto_mes, mes_1 si esta en la foto_mes siguiente y mes_2 si esta en las siguientes dos foto_mes
Debe completar su lógica en la variable clase_ternaria usando estas variables auxiliares
La tabla final se queda solamente con las variables y los registros necesarios."


#### Lógica que utilicé (Isma): Si mes_0 = 1, mes_1 = 0, mes_2 = 0  entonces BAJA +1 / Si mes_0=1, mes_1=1, mes_2 = 0, entonces BAJA+2 / Si mes_0=1, mes_1 = 0 , mes_2= 1 entonces CONTINUA / los demas casos son CONTINUA

In [9]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , CASE
            WHEN mes_0 = 1 AND mes_1 = 0 AND mes_2 = 0 THEN 'BAJA+1'
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2 = 0 THEN 'BAJA+2'
            WHEN mes_0 = 1 AND mes_1 = 0 AND mes_2 = 1 THEN 'CONTINUA'
            ELSE 'CONTINUA'
          END as clase_ternaria
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


"Luego agregamos la variable clase_ternaria a nuestra tabla y cargamos los datos que previamente calculamos."


In [10]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)


,Success


In [11]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes


,Success


"Y finalmente guardamos el dataset para usarlo en la competencia. Esto les va a tardar en guardar."


In [12]:
%%sql
copy competencia_02 to '/content/competencia_02.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


#### Importante: Hay que descargarse el archivo "competencia_02.csv.gz" generado y subirlo al bucket de google cloud

##### Para chequear que esté bien creada clase_ternaria

In [4]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('/content/competencia_02.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [13]:
%%sql
SELECT * FROM competencia_02 WHERE clase_ternaria IS NOT NULL LIMIT 200

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8.0,0.00,-35773.32,2.33,5148.0,36602.13,54.0,0.0,5008.71,CONTINUA
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,1.0,0.00,-67414.82,1149.57,3299.0,28503.06,10.0,0.0,1395.87,CONTINUA
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,1.0,0.00,0.00,0.00,6790.0,5170.25,2.0,0.0,0.00,CONTINUA
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,22.0,69653.59,-59422.00,0.00,7037.0,67991.90,31.0,0.0,2897.31,CONTINUA
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,8.0,0.00,-4802.21,0.00,7596.0,4851.93,3.0,0.0,492.66,CONTINUA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,29297645,201901,1,0,1,39,249,1330.78,12139.81,1028.07,...,1.0,0.00,-9703.58,0.00,4259.0,7501.51,6.0,0.0,0.00,CONTINUA
196,29298063,201901,1,1,1,66,296,19137.76,171920.83,6669.78,...,1.0,0.00,-6079.36,0.00,9601.0,1291.90,5.0,0.0,164.22,CONTINUA
197,29298226,201901,1,0,0,57,242,880.28,7856.84,726.27,...,22.0,4052.02,-1232.02,0.00,2438.0,1758.32,1.0,0.0,187.68,CONTINUA
198,29298618,201901,1,0,1,54,35,1604.96,14029.34,726.13,...,8.0,11157.05,-4832.23,0.00,1045.0,6829.44,7.0,0.0,469.20,CONTINUA
